In [ ]:
import requests
import pandas as pd

In [ ]:
API_url = 'http://stats.oecd.org/SDMX-JSON/data/PAT_DEV/all/all'

r = requests.get(API_url, )

In [ ]:
json_r = r.json()

In [4]:
structure = json_r['structure']

structure_dims = structure['dimensions']

def parse_structure_dims(structure_dims):
    
    dfs = {}
    
    for dim in structure_dims['series']:
        
        key = f"key_{dim['keyPosition']}"
        df = pd.DataFrame(dim['values']).rename(columns={'id': dim['id'], 'name': dim['name']}).reset_index().rename(columns={'index':key})
        
        dfs[key] = df
    
    for obs in structure_dims['observation']:
        
        dfs[obs['role']] = pd.DataFrame(obs['values']).rename(columns={'id': obs['id'], 'name': obs['name']})

    return dfs
    
    
parsed_dims = parse_structure_dims(structure_dims)

In [5]:
dataSets = json_r['dataSets']

In [6]:
series = dataSets[0]['series']

In [7]:
def parse_key(key):
    split = key.split(':')
    return {f'key_{i}': int(k) for i,k in enumerate(split)}

In [16]:
def parse_series(series):
    
    dfs = []
    
    for serie_key, serie in series.items():
        key_idx = parse_key(serie_key)
        
        df = pd.DataFrame.from_dict(serie['observations'], orient='index').rename(columns = {0: 'Value'})
        df.index = df.index.astype(int)
        
        for key, idx in key_idx.items():
            df[key] = idx
        
        dfs.append(df)  
    
    return pd.concat(dfs)

In [17]:
series_df = parse_series(series)

In [14]:
parsed_dims['TIME_PERIOD']

RangeIndex(start=0, stop=59, step=1)

In [10]:
series_df.merge(parsed_dims['TIME_PERIOD'], left_index=True, right_index=True)

,Value,1,key_0,key_1,key_2,TIME_PERIOD,Year
55,0.5,None,210,1,185,2015,2015
57,0.5,None,210,1,185,2017,2017


In [ ]:
(
    test.merge(parsed_dims['key_0'], on='key_0')
        .merge(parsed_dims['key_1'], on='key_1')
        .merge(parsed_dims['key_2'], on='key_2')
)
